In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/train.csv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/antecedents.tsv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/consequents.tsv /content
#!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/simplified.tsv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/outputs.antecedents/submit_results.csv /content/antecedent_results.csv
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/outputs.consequents/submit_results.csv /content/consequent_results.csv

In [ ]:
import pandas as pd

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None) # change to test.csv
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1]
})
# display 
test_df.head()

In [ ]:
import pandas as pd

prefix = '/content/'

a_df = pd.read_csv(prefix + 'antecedents.tsv', sep='\t', delimiter=None, header=None)
a_df = pd.DataFrame({
    'id': a_df[0],
    'labels':a_df[1],
    'text': a_df[2],
    'start':a_df[3],
    'end': a_df[4]
})
#a_df.sort_values(by=['id','text'], ascending=[1,0])

c_df = pd.read_csv(prefix + 'consequents.tsv', sep='\t', delimiter=None, header=None)
c_df = pd.DataFrame({
    'id': c_df[0],
    'labels':c_df[1],
    'text': c_df[2],
    'start':c_df[3],
    'end': c_df[4]
})

rc_df = pd.read_csv(prefix + 'consequent_results.csv', header=None)
rc_df = pd.DataFrame({
    'id': rc_df[0],
    'labels':rc_df[1]
})
rc_df.sort_values(by=['labels'], ascending=False, inplace = True)

ra_df = pd.read_csv(prefix + 'antecedent_results.csv', header=None)
ra_df = pd.DataFrame({
    'id': ra_df[0],
    'labels':ra_df[1]
})
ra_df.sort_values(by=['labels'], ascending=False, inplace = True)
ra_df.head(10)

In [ ]:
pip install allennlp==1.0.0 allennlp-models==1.0.0

In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.structured_prediction
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz")

original = 'Oh, the military could have swooped in to save the day, they say, if only they had not been prevented from doing that by Col.'    
prediction = predictor.predict(
  sentence = original
)
prediction

In [ ]:
def extract_dependents(prediction):
 root = None
 text = str()

 for k, v in prediction.items():
  if (k=='hierplane_tree'):
    for key, val in v.items():
      if key=='root':
        root = val
      if key=='text': 
        text = val   

 root.update({'text': text})
 return extract_children(root) 

In [ ]:
def extract_children(obj, include='', exclude=''):
  arr = list()
  deps = list()
  spans = list()
  verbs = list()

  def extract(obj, arr): 
    explore = False
    limit = 5 
    count = 0
    text = str()

    if isinstance(obj, dict):
      for k, v in obj.items():
        if include =='' and k=='attributes' and ('VERB' in v or 'ADJ' in v or 'AUX' in v or 'PROPN' in v or 'PREP' in v or 'NOUN' in v):
          if obj not in verbs:
            verbs.append(obj)

        if k=='nodeType':
          arr.append(v)

        if k=='text' and len(v) > 0 and len(text) == 0:  
          text = v

        if isinstance(v, (dict, list)):
          if count < limit:
            count = count + 1
            extract(v, arr) # children   
             
    elif isinstance(obj, list):
      for item in obj:
        if isinstance(item, dict):
    
          for key, value in item.items():
            if key == 'nodeType':
              if (include=='' or value in include) and (exclude=='' or not [v for v in arr if v in exclude]):
                #print(include, exclude, item)
                if item not in deps: 
                  deps.append(item) # child
                explore = True 
              else:  
                explore = False

            if include=='' and key=='attributes' and ('VERB' in value or 'ADJ' in value or 'AUX' in value or 'PROPN' in value or 'PREP' in value or 'NOUN' in value):
              explore = True 
            elif key=='attributes' and 'CCONJ' in value:
              explore = False     

            if explore and key =='spans':
              if value not in spans:
                spans.append(value)  
            
          if explore and count < limit:
            count = count + 1
            extract(item, arr) 
         
    return text

  text = extract(obj, arr)
  return text, spans, deps, verbs

In [ ]:
def position(span):
  return span[0]['start']

In [ ]:
def follow_children(obj):
  arr = list()

  def follow(obj, arr): 

    if isinstance(obj, dict):
      for k, v in obj.items():
        if k == 'children':      
          follow(v, arr)

    elif isinstance(obj, list):
      for item in obj:
        if isinstance(item, dict):
          for key, value in item.items():
            if key =='attributes' and ('VERB' in value or 'ADJ' in value or 'AUX' in value or 'PROPN' in value or 'PREP' in value or 'NOUN' in value):
              arr.append(item)

          follow(item, arr)
      
    return arr

  results = follow(obj, arr)
  return results

In [ ]:
import re
regex = '\\\'\s+([^\']+)\s+\\\''
def compact(fragment):
# specify the number of replacements by changing the 4th argument
  result = re.sub(regex, '\''+r'\1'+'\'' , fragment)
  return result

In [ ]:
def normalize(fragment):
  r = fragment.strip()
  r = compact(r)
  r = r.replace('   ',' ') 
  r = r.replace('  ',' ')
  r = r.replace(' ,',',') 
  r = r.replace(' .','.')
  r = r.replace('Mr.','Mr. ')
  r = r.replace('Ms.','Ms. ')
  r = r.replace('Mrs.','Mrs. ')
  r = r.replace('.  ','. ') # Mr./Ms./Mrs.
  r = r.replace('... ','...')
  r = r.replace(' --- ','---')
  r = r.replace(' -- ','--')
  r = r.replace(' - ','-')
  r = r.replace(' !','!')
  r = r.replace(' ?','?')
  r = r.replace('$ ','$')
  r = r.replace('# ','#')
  r = r.replace(' %','%')
  r = r.replace('( ','(')
  r = r.replace(' )',')')
  r = r.replace(' ;',';')
  r = r.replace(' :',':') 
  r = r.replace(': ',':') #50/50 - carefull!
  r = r.replace(' nt','nt')
  r = r.replace(' (k)','(k)') #401(k)
  r = r.replace(' (tm)','(tm)')
  r = r.replace('n na ','nna ')
  r = r.replace(' \'m ','\'m ')
  r = r.replace('s \' ','s\' ') # was 'neurtralized'?
  r = r.replace(' \'s ','\'s ') # a 'surgical' strike!
  r = r.replace(' \'d ','\'d ')
  r = r.replace(' \'re ','\'re ')
  r = r.replace(' \'ve ','\'ve ')
  r = r.replace(' \'ll ','\'ll ')
  r = r.replace(' n\'t','n\'t') # what about UPPER()?
  r = r.replace(' \' ',' \'')
  r = r.rstrip()
  return r

In [ ]:
def explore(verb, condition, complement, sentence, include='',exclude=''):
    deps = list()
    loops= list()
    if len(condition)==0:
      return deps, loops
    min=max=begin=end=-1 

    begin = sentence.find(condition) # superspan for condition
    if begin ==-1: 
      #print(condition,' not in:\n'+sentence)
      return deps, loops
    stop = begin + len(condition) + 1 # space
    
    if len(complement) > 0:
      min = sentence.find(complement) # superspan for complement
      if min ==-1: 
        #print(complement,' not in:\n'+sentence)
        return deps,loops
      max = min + len(complement) + 1 # space  
      #print(min,complement,max)

    word = verb['word']
    #print( word ,'-->', condition)
    subtext, subspans, subjs, subverbs = extract_children(verb, include, exclude)
    if len(subspans) == 0:
      return deps, loops

    for sub in subjs:    
      descendant = sub['word']
      #print(word, begin, '\"'+descendant+'\"', stop) 
      for span in sorted(subspans,key=position): # all subspans
        start=span[0]['start']
        end = span[0]['end']
        if start > 2: start=start - 2 
        if end > 2: end = end - 2 # due to some unknown issue?!?
        spantext = sentence[start:end]

        if len(spantext) > 0:
          if begin <= start and start < stop: # span of the condition
            if spantext in condition and spantext not in deps:
              #print(spantext, start, end, condition)
              deps.append(spantext)     

            if min <= start and start < max: # span of the complement
              if spantext in complement and spantext not in loops: 
                #print(spantext, start, end, complement)
                loops.append(spantext)

    return deps, loops       

In [ ]:
def track(verbs, antecedent, consequent, sentence):
  adeps = set()
  aloops= set()
  cdeps = set()
  cloops= set()
  if len(verbs)==0:
    print('no verbs!')
    return adeps, cdeps, aloops, cloops

  for verb in verbs: # any 'VERB' or "AUX" or "NOUN", because those are typicaly at root
    deps, loops = explore(verb, antecedent, consequent, sentence, 'dep nsubj conj ccomp','cc advcl')
    adeps.update(deps)
    aloops.update(loops)
    deps, loops = explore(verb, consequent, antecedent, sentence, 'dep prep advmod aux conj ccomp','cop xcomp')
    cdeps.update(deps)
    cloops.update(loops)
    if len(adeps) > 0 and len(cdeps) > 0:
      return adeps, cdeps, aloops, cloops    
    adeps = set()
    cdeps = set()  

  return adeps, cdeps, aloops, cloops   

In [ ]:
def separate_from(md, pa):
  index = -1
  if pa in md and len(md) > len(pa):
    index = md.find(pa)
    if index > 0 :
      md = md[0:index]
      #print('|'+md+'{...}')
    elif index == 0:
      md = md.replace(pa,'')
      #print('|{...}'+md)  
  return md, index       

In [ ]:
def analyze(id, df, r_df, sentence, verbs=[], ctext='', first=True):
  atext = str()
  astart=-1
  aend = -1

  apred = r_df.loc[r_df.id.str.contains(id)]
  if len(apred)==0:
    return id, 0, atext, astart, aend, 0

  print(first,'_',ctext)
  # check based on rules
  #print(id,len(arule.index))
 
  alab ='0' # assume antecedent false
  count= 0  # counter of predictions
  for p in apred.index: # [p] = index
    count = count + 1
    amid = apred['id'][p]  
    alab = apred['labels'][p] #predicted

    arule = df.loc[df.id.str.contains(amid)]
    assert len(arule) > 0

    for a in arule.index: # [a] = index 
      ralab = arule['labels'][a] 
      astart = arule['start'][a]
      aend = arule['end'][a]
      atext = arule['text'][a]
      text = atext
      
      print('|', alab, atext)
      if first:
        adeps,cdeps,aloops,cloops = track(verbs, atext, ctext, sentence)
      else: 
        adeps,cdeps,aloops,cloops = track(verbs, ctext, atext, sentence)
      print(first,adeps,cdeps,aloops,cloops) # eg. 'set() set() set() set()'

      if ralab == 1:      
        if not first or (len(adeps) > 0 and len(cdeps) > 0):
          if len(aloops) > 0 or len(cloops) > 0:
            if len(cloops)>0:
              atext = atext + ' ' + ctext
              aend = aend + 1 + len(ctext)
            else:
              atext = ctext + ' ' + atext
              astart = aend-1 - len(ctext)
            print('+' , atext)   
            return amid, alab, atext, astart, aend,'-50' # continue to next step

        print('V')
        return amid, alab, atext, astart, aend, 100 # no other options available
      else:
        if len(adeps) >0 and len(cdeps) >0 and len(aloops) >0 and len(cloops) >0:
          if len(apred.index) > count:
            print('x')
            continue    
          return amid, alab, atext, astart, aend, '-100' # give up and fail

        if alab == 1:  
          if not first or (len(adeps) > 0 and len(cdeps) > 0):
            if len(aloops) > 0 or len(cloops) > 0:
              if len(cloops)>0:
                atext = atext + ' ' + ctext
                aend = aend + 1 + len(ctext)
              else:
                atext = ctext + ' ' + atext
                astart = aend-1 - len(ctext) 
              print('+' , atext)   
              return amid, alab, atext, astart, aend,'-50' # continue to next step

          print('v')
          return amid, alab, atext, astart, aend, 75
        else:
          
          if len(aloops) > 0 or len(cloops) > 0:
            if first:
                print('^',count)
                return amid, alab, atext, astart, aend,'-25'
            else:
                if len(cloops)>0:
                  atext = atext + ' ' + ctext
                  aend = aend + 1 + len(ctext)
                else:
                  atext = ctext + ' ' + atext
                  astart = aend-1 - len(ctext)
                print('+' , atext)   
                return amid, alab, atext, astart, aend,'-50' 

          print('v')              
          return amid, alab, atext, astart, aend, 50 # uncertain     
  print('v')
  return amid, alab, atext, astart, aend, 25 # unpredicted

In [ ]:
original = '"Honestly, if the Democrats would have won the election, first of all, you would have had a lot different - if you look at the last four decisions in the Supreme Court at 5-4, they would have all been reversed," he said.'
prediction = predictor.predict(sentence = original)
sentence, spans, deps, verbs = extract_dependents(prediction)

verbs = follow_children(verbs) # flatten the tree
#print(verbs)
antecedent = "if the Democrats would have won the election"
consequent = "would have"
sentence = normalize(sentence)

adeps, cdeps, aloops, cloops = track(verbs, antecedent, consequent, sentence)
print(adeps,cdeps)
print(aloops,cloops)

In [ ]:
import pandas as pd

from tqdm import tqdm
from time import sleep

submit_df = pd.DataFrame(columns=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid'])
report_df = pd.DataFrame(columns=['sentenceID','clabel','cprob','cdep','consequent','gold_consequent','cstart','cend','gold_antecedent','antecedent', 'alabel','aprob','astart','aend'])

count = 0
for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]
  id = test_df['id'][ind]

  original = original.replace('\"','')
  prediction = predictor.predict(original)

  rebuilt, spans, deps, verbs = extract_dependents(prediction)
  verbs = follow_children(verbs) # flatten the   
  sentence = normalize(rebuilt)
  print(id, sentence)
  
  astart=-1
  aend = -1
  cstart=-1
  cend = -1
  text =str()
  atext=str()  
  ctext=str()

  cid, clab, ctext, cstart, cend, cprob = analyze(id, c_df, rc_df, sentence, verbs, atext)
  aid, alab, atext, astart, aend, aprob = analyze(id, a_df, ra_df, sentence, verbs, ctext)
  print(aprob,'A')

  if aprob != '-100' and aprob != '-50' and aprob!='-25':
    cid, clab, ctext, cstart, cend, cprob = analyze(id, c_df, rc_df, sentence, verbs, atext)
  if aprob == '-50':
    ctext = atext
    aid, alab, atext, astart, aend, aprob = analyze(id, a_df, ra_df, sentence, verbs, ctext, False)
  if aprob == '-25':
    cid,clab,ctext, cstart, cend, cprob = analyze(id, c_df, rc_df, sentence, verbs, atext, False)
  print(cprob,'C')  

  if cprob != 100 and cprob != 75 and cprob != 50 and cprob != 25:
    aid, alab, atext, astart, aend, aprob = analyze(id, a_df, ra_df, sentence, verbs, ctext)
  print(aprob,'A')  

  consequent = ctext
  antecedent = atext
  #print(antecedent)
  #print(consequent)

  cdep = '' # Pass
  if consequent == antecedent:
    consequent = ''
    cstart=-1
    cend = -1
    cdep = '='

  if len(consequent) > 0:
    if consequent in antecedent:
      consequent = ''
      cstart=-1
      cend = -1
      cdep = '<'
    elif antecedent in consequent:
      antecedent = consequent
      astart=cstart
      aend = cend
      aprob= cprob
      consequent = ''
      cstart=-1
      cend = -1
      cdep = '>'
    
  if len(antecedent) > 0:       
    if len(consequent) > 0:
      text = antecedent + ' | ' + consequent
    else:
      text = antecedent
  else:
    cdep = '!'      

  print(cdep) # post-correction
  if len(cdep) > 0:
    count=count+1

  report_row = pd.Series( data={
    'sentenceID': id,
    'clabel': clab,
    'cprob': cprob,
    'cdep' : cdep,
    'consequent': consequent, # comment out gold_
    #'gold_consequent': test_df['consequent'][ind],
    'cstart': cstart,
    'cend': cend,
    #'gold_antecedent': test_df['antecedent'][ind],
    'antecedent': antecedent, 
    'alabel': alab,
    'aprob': aprob,
    'astart': astart,
    'aend': aend
  }, name = str(id) )

  result_row = pd.Series( data={
    'sentenceID': id,
    'antecedent_startid':astart,
    'antecedent_endid': aend,
    'consequent_startid': cstart,
    'consequent_endid': cend
  }, name = str(id) )

  submit_df = submit_df.append(result_row, ignore_index=True)
  report_df = report_df.append(report_row, ignore_index=True)

  print(id, text, astart, aend, cstart, cend)
  sleep(1)

submit_df.to_csv(prefix+'subtask2.csv',index=False, header=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid']) 
report_df.to_csv(prefix+'report.csv',index=False, header=['sentenceID','clabel','cprob','cdep','consequent','gold_consequent','cstart','cend','gold_antecedent','antecedent','alabel','aprob','astart','aend'])
print('Adjusted =',count)

In [ ]:
!zip submission_task2.zip subtask2.csv
!mv /content/submission_task2.zip /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/report.csv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/